In [3]:
# ---------------------------------------------------------
# SCRIPT: sandbox.py
# DESCRIPCIÓN: Estudio de los datos y su estructura
# ---------------------------------------------------------

from pyspark.sql import SparkSession

# Configuramos la sesión (el "driver") para que sepa hablar con MinIO
print("🔌 Conectando Spark con el Data Lake...")

spark = SparkSession.builder \
    .appName("SkyTracker_Exploracion") \
    .master("local[*]") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark está listo para recibir órdenes.")

🔌 Conectando Spark con el Data Lake...
✅ Spark está listo para recibir órdenes.


In [4]:
# Leemos el archivo tal cual es (Raw)
print("📂 Leyendo 'flights.csv' desde la capa Bronze...")

df_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://bronze/raw_data/flights.csv")

# Imprimimos la estructura 
print("\n--- 📋 ESQUEMA DE DATOS (COLUMNAS Y TIPOS) ---")
df_raw.printSchema()

# Vemos un ejemplo real (las primeras 2 filas) para entender qué hay dentro
print("\n--- 👀 VISTA PREVIA DE LOS DATOS ---")
df_raw.show(2, vertical=True) 

📂 Leyendo 'flights.csv' desde la capa Bronze...

--- 📋 ESQUEMA DE DATOS (COLUMNAS Y TIPOS) ---
root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (

In [5]:
# --- SELECCIÓN DE COLUMNAS---
print("🧹 Seleccionando solo la información ...")

df_bronze = df_raw.select(
    "YEAR", 
    "MONTH", 
    "DAY",
    "DAY_OF_WEEK",
    "AIRLINE", 
    "FLIGHT_NUMBER",
    "TAIL_NUMBER",
    "ORIGIN_AIRPORT", 
    "DESTINATION_AIRPORT",
    "SCHEDULED_DEPARTURE",
    "DEPARTURE_TIME",
    "DEPARTURE_DELAY",
    "ARRIVAL_DELAY",
    "DISTANCE",
    "AIR_TIME",
    "CANCELLED",
    "CANCELLATION_REASON"
)

print("✅ DataFrame 'df_bronze' creado.")
print("--- Muestra de las primeras 5 filas ---")
df_bronze.show(2, vertical=True)

🧹 Seleccionando solo la información ...
✅ DataFrame 'df_bronze' creado.
--- Muestra de las primeras 5 filas ---
-RECORD 0---------------------
 YEAR                | 2015   
 MONTH               | 1      
 DAY                 | 1      
 DAY_OF_WEEK         | 4      
 AIRLINE             | AS     
 FLIGHT_NUMBER       | 98     
 TAIL_NUMBER         | N407AS 
 ORIGIN_AIRPORT      | ANC    
 DESTINATION_AIRPORT | SEA    
 SCHEDULED_DEPARTURE | 5      
 DEPARTURE_TIME      | 2354   
 DEPARTURE_DELAY     | -11    
 ARRIVAL_DELAY       | -22    
 DISTANCE            | 1448   
 AIR_TIME            | 169    
 CANCELLED           | 0      
 CANCELLATION_REASON | NULL   
-RECORD 1---------------------
 YEAR                | 2015   
 MONTH               | 1      
 DAY                 | 1      
 DAY_OF_WEEK         | 4      
 AIRLINE             | AA     
 FLIGHT_NUMBER       | 2336   
 TAIL_NUMBER         | N3KUAA 
 ORIGIN_AIRPORT      | LAX    
 DESTINATION_AIRPORT | PBI    
 SCHEDULED_DEPARTURE

In [6]:
print("--- ✈️ INSPECCIONANDO DIMENSIONES ---")

# Cargar Aerolíneas (Diccionario de compañías)
df_airlines_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://bronze/raw_data/airlines.csv")

print("\n--- 1. AEROLÍNEAS (Schema & Data) ---")
df_airlines_raw.printSchema()
df_airlines_raw.show(5, truncate=False) 

# Cargar Aeropuertos (Diccionario de lugares + Coordenadas)
df_airports_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://bronze/raw_data/airports.csv")

print("\n--- 2. AEROPUERTOS (Schema & Data) ---")
df_airports_raw.printSchema()
df_airports_raw.show(5)

--- ✈️ INSPECCIONANDO DIMENSIONES ---

--- 1. AEROLÍNEAS (Schema & Data) ---
root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)

+---------+----------------------+
|IATA_CODE|AIRLINE               |
+---------+----------------------+
|UA       |United Air Lines Inc. |
|AA       |American Airlines Inc.|
|US       |US Airways Inc.       |
|F9       |Frontier Airlines Inc.|
|B6       |JetBlue Airways       |
+---------+----------------------+
only showing top 5 rows


--- 2. AEROPUERTOS (Schema & Data) ---
root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)

+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+---------

In [7]:
# --- CREACIÓN DE DIMENSIONES LIMPIAS ---

# Aerolíneas: Nos quedamos con todo (Código y Nombre)
print("🧹 Limpiando tabla Aerolíneas...")
df_airlines = df_airlines_raw.select(
    df_airlines_raw["IATA_CODE"].alias("AIRLINE_ID"), # Renombramos para evitar confusiones
    df_airlines_raw["AIRLINE"].alias("AIRLINE_NAME")
)

# Aeropuertos: Quitamos COUNTRY
print("🧹 Limpiando tabla Aeropuertos (Eliminando 'Country')...")
df_airports = df_airports_raw.select(
    "IATA_CODE",
    "AIRPORT",
    "CITY",
    "STATE",
    "LATITUDE",
    "LONGITUDE"
)


🧹 Limpiando tabla Aerolíneas...
🧹 Limpiando tabla Aeropuertos (Eliminando 'Country')...


In [8]:
# PRUEBAS CAPA SILVER

# Cargar los DATOS OPTIMIZADOS (Parquet) de la capa Bronze
print("📂 Cargando tablas Bronze (Formatted)...")
df_vuelos = spark.read.parquet("s3a://bronze/formatted/flights")
df_airlines = spark.read.parquet("s3a://bronze/formatted/airlines")
df_airports = spark.read.parquet("s3a://bronze/formatted/airports")
print("✅ Tablas listas en memoria:")


📂 Cargando tablas Bronze (Formatted)...
✅ Tablas listas en memoria:


In [9]:
print("\n📋 Esquema de VUELOS (df_vuelos):")
df_vuelos.printSchema()

print("\n📋 Esquema de AEROLÍNEAS (df_airlines):")
df_airlines.printSchema()

print("\n📋 Esquema de AEROPUERTOS (df_airports):")
df_airports.printSchema()


📋 Esquema de VUELOS (df_vuelos):
root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- CANCELLATION_REASON: string (nullable = true)


📋 Esquema de AEROLÍNEAS (df_airlines):
root
 |-- AIRLINE_ID: string (nullable = true)
 |-- AIRLINE_NAME: string (nullable = true)


📋 Esquema de AEROPUERTOS (df_airports):
root
 |-- IATA_CODE: string (nullable =

In [10]:
# Unir flights con airlines: "left" significa: Quédate con todos los vuelos, y si encuentras la aerolínea, pega su nombre.
df_step1 = df_vuelos.join(
    df_airlines, 
    df_vuelos["AIRLINE"] == df_airlines["AIRLINE_ID"], 
    "left"
)

# Limpieza: Ahora tenemos 'AIRLINE' (código) y 'AIRLINE_ID' (código repetido): Borramos la repetida para no ensuciar.
df_step1 = df_step1.drop("AIRLINE_ID")

print("✅ Paso 1 completado. Muestra con nombre de aerolínea:")
df_step1.select("FLIGHT_NUMBER", "AIRLINE", "AIRLINE_NAME").show(5)

✅ Paso 1 completado. Muestra con nombre de aerolínea:
+-------------+-------+--------------------+
|FLIGHT_NUMBER|AIRLINE|        AIRLINE_NAME|
+-------------+-------+--------------------+
|           98|     AS|Alaska Airlines Inc.|
|         2336|     AA|American Airlines...|
|          840|     US|     US Airways Inc.|
|          258|     AA|American Airlines...|
|          135|     AS|Alaska Airlines Inc.|
+-------------+-------+--------------------+
only showing top 5 rows



In [11]:
# Unir flights con airports: el problema es que tenemos que unir dos columnas, por lo que hay variables que se nos duplicarán
from pyspark.sql.functions import col

# ORIGEN
# Usamos .alias("origen") para decirle a Spark: "Trata esta copia de aeropuertos como 'origen'"
print("📍 Cruzando datos del Aeropuerto de ORIGEN...")

df_step2 = df_step1.join(
    df_airports.alias("origen"), 
    col("ORIGIN_AIRPORT") == col("origen.IATA_CODE"),
    "left"
)

# DESTINO
# Usamos .alias("destino") para la segunda vez
print("📍 Cruzando datos del Aeropuerto de DESTINO...")

df_silver = df_step2.join(
    df_airports.alias("destino"), 
    col("DESTINATION_AIRPORT") == col("destino.IATA_CODE"),
    "left"
)

print("✅ Joins completados, mostrando pruebas.")
df_silver.select(
    "FLIGHT_NUMBER",
    "ORIGIN_AIRPORT",
    col("origen.CITY").alias("CIUDAD_ORIGEN"),    # Accedemos usando el alias
    col("destino.CITY").alias("CIUDAD_DESTINO"),  # Accedemos usando el alias
    col("origen.LATITUDE").alias("LAT_ORIGEN"),
    col("destino.LATITUDE").alias("LAT_DESTINO")
).show(5)

📍 Cruzando datos del Aeropuerto de ORIGEN...
📍 Cruzando datos del Aeropuerto de DESTINO...
✅ Joins completados, mostrando pruebas.
+-------------+--------------+-------------+---------------+----------+-----------+
|FLIGHT_NUMBER|ORIGIN_AIRPORT|CIUDAD_ORIGEN| CIUDAD_DESTINO|LAT_ORIGEN|LAT_DESTINO|
+-------------+--------------+-------------+---------------+----------+-----------+
|           98|           ANC|    Anchorage|        Seattle|  61.17432|   47.44898|
|         2336|           LAX|  Los Angeles|West Palm Beach|  33.94254|   26.68316|
|          840|           SFO|San Francisco|      Charlotte|    37.619|   35.21401|
|          258|           LAX|  Los Angeles|          Miami|  33.94254|   25.79325|
|          135|           SEA|      Seattle|      Anchorage|  47.44898|   61.17432|
+-------------+--------------+-------------+---------------+----------+-----------+
only showing top 5 rows



In [15]:
# CAPA GOLD
print("📂 Cargando Silver Master...")
df_silver = spark.read.parquet("s3a://silver/master_flights")
print("📋 ESTRUCTURA DE LA CAPA SILVER (Master Table):")
df_silver.printSchema()

# KPI 1: HEATMAP DE FIABILIDAD
from pyspark.sql.functions import avg, count, round, col, when

# 3. KPI 1 MEJORADO: Tratando los adelantos como 0
print("🔥 Generando KPI 1 (Los adelantos cuentan como 0 retraso)...")

df_heatmap = df_silver.groupBy("DAY_NAME", "DEPARTURE_HOUR") \
    .agg(
        # Si es negativo (< 0), cuenta como 0. Si no, usa el valor real.
        round(
            avg(
                when(col("DEPARTURE_DELAY") < 0, 0)
                .otherwise(col("DEPARTURE_DELAY"))
            ), 2
        ).alias("RETRASO_MEDIO"),
        
        count("*").alias("NUM_VUELOS")
    ) \
    .orderBy("DAY_NAME", "DEPARTURE_HOUR")

print("✅ Tabla Gold KPI 1 (Fiable) lista:")
df_heatmap.show(10, truncate=False)

📂 Cargando Silver Master...
📋 ESTRUCTURA DE LA CAPA SILVER (Master Table):
root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- DAY_NAME: string (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- AIRLINE_NAME: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- ORIGIN_CODE: string (nullable = true)
 |-- ORIGIN_NAME: string (nullable = true)
 |-- ORIGIN_CITY: string (nullable = true)
 |-- ORIGIN_STATE: string (nullable = true)
 |-- ORIGIN_LAT: double (nullable = true)
 |-- ORIGIN_LONG: double (nullable = true)
 |-- DEST_CODE: string (nullable = true)
 |-- DEST_NAME: string (nullable = true)
 |-- DEST_CITY: string (nullable = true)
 |-- DEST_STATE: string (nullable = true)
 |-- DEST_LAT: double (nullable = true)
 |-- DEST_LONG: double (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable 

In [16]:
# KPI 2: ESTRES EN LA RED DE RUTAS: Numero de vuelos y retraso 
from pyspark.sql.functions import col, avg, count, round, first, when
print("🌍 Generando KPI 2: Mapa de Rutas...")

df_routes = df_silver.groupBy("ORIGIN_CODE", "DEST_CODE") \
    .agg(
        # --- A. VUELOS POR CADA RUTA ---
        count("*").alias("TOTAL_VUELOS"),
        
        # Lógica de "Adelantos no restan" que en el KPI 1
        round(
            avg(
                when(col("DEPARTURE_DELAY") < 0, 0)
                .otherwise(col("DEPARTURE_DELAY"))
            ), 2
        ).alias("RETRASO_MEDIO"),
        
        # --- B. RESCATAR COORDENADAS Y NOMBRES (GEO) ---
        # Como agrupamos por Código, usamos 'first' para recuperar el nombre y la posición
        first("ORIGIN_CITY").alias("ORIGEN_CIUDAD"),
        first("DEST_CITY").alias("DESTINO_CIUDAD"),
        
        first("ORIGIN_LAT").alias("LAT_ORIGEN"),
        first("ORIGIN_LONG").alias("LONG_ORIGEN"),
        
        first("DEST_LAT").alias("LAT_DESTINO"),
        first("DEST_LONG").alias("LONG_DESTINO")
    )

# 3. FILTRADO (Limpieza de Ruido)
# Quitamos rutas que tengan menos de 50 vuelos al año (son irrelevantes para el mapa general)
df_routes = df_routes.filter(col("TOTAL_VUELOS") >= 50)

# 4. ORDENAR (Las rutas más transitadas primero)
df_routes = df_routes.orderBy(col("TOTAL_VUELOS").desc())

print("✅ Tabla Gold KPI 2 (Rutas) lista:")
df_routes.show(10)

🌍 Generando KPI 2: Mapa de Rutas...
✅ Tabla Gold KPI 2 (Rutas) lista:
+-----------+---------+------------+-------------+-------------+--------------+----------+-----------+-----------+------------+
|ORIGIN_CODE|DEST_CODE|TOTAL_VUELOS|RETRASO_MEDIO|ORIGEN_CIUDAD|DESTINO_CIUDAD|LAT_ORIGEN|LONG_ORIGEN|LAT_DESTINO|LONG_DESTINO|
+-----------+---------+------------+-------------+-------------+--------------+----------+-----------+-----------+------------+
|        SFO|      LAX|       13744|        14.83|San Francisco|   Los Angeles|    37.619| -122.37484|   33.94254|  -118.40807|
|        LAX|      SFO|       13457|        16.12|  Los Angeles| San Francisco|  33.94254| -118.40807|     37.619|  -122.37484|
|        JFK|      LAX|       12016|        10.59|     New York|   Los Angeles|  40.63975|  -73.77893|   33.94254|  -118.40807|
|        LAX|      JFK|       12015|        13.14|  Los Angeles|      New York|  33.94254| -118.40807|   40.63975|   -73.77893|
|        LAS|      LAX|        971

In [17]:
# KPI 3: LOS VUELOS QUE SALEN TARDE, CUANTO SON CAPACES DE RECUPERAR. AGRUPAR POR EROLINEAS
from pyspark.sql.functions import avg, count, round, col

print("🚀 Generando KPI 3: Eficiencia de Pilotos (Solo vuelos con retraso inicial)...")

# 1. FILTRADO: Solo vuelos que salieron TARDE (> 0) y no cancelados
df_retrasados = df_silver.filter(
    (col("CANCELLED") == 0) & 
    (col("DEPARTURE_DELAY") > 0) &  # ¡AQUÍ ESTÁ TU CAMBIO! Solo los positivos
    (col("ARRIVAL_DELAY").isNotNull())
)

# 2. AGRUPACIÓN Y CÁLCULO
df_pilots = df_retrasados.groupBy("AIRLINE_NAME") \
    .agg(
        count("*").alias("VUELOS_RETRASADOS"), # Cuántas veces salieron tarde
        
        # Retraso promedio AL SALIR (de los que salieron tarde)
        round(avg("DEPARTURE_DELAY"), 2).alias("AVG_RETRASO_INICIAL"),
        
        # Retraso promedio AL LLEGAR
        round(avg("ARRIVAL_DELAY"), 2).alias("AVG_RETRASO_FINAL"),
        
        # TIEMPO RECUPERADO = Lo que tenía al salir - Lo que tengo al llegar
        round(
            avg(col("DEPARTURE_DELAY") - col("ARRIVAL_DELAY")), 2
        ).alias("TIEMPO_RECUPERADO_AVG")
    )

# 3. ORDENAR: Los que más tiempo recuperan (número más alto) arriba
df_pilots = df_pilots.orderBy(col("TIEMPO_RECUPERADO_AVG").desc())

print("✅ Tabla Gold KPI 3 (Pilotos) lista:")
df_pilots.show(20, truncate=False)

🚀 Generando KPI 3: Eficiencia de Pilotos (Solo vuelos con retraso inicial)...
✅ Tabla Gold KPI 3 (Pilotos) lista:
+----------------------------+-----------------+-------------------+-----------------+---------------------+
|AIRLINE_NAME                |VUELOS_RETRASADOS|AVG_RETRASO_INICIAL|AVG_RETRASO_FINAL|TIEMPO_RECUPERADO_AVG|
+----------------------------+-----------------+-------------------+-----------------+---------------------+
|United Air Lines Inc.       |255368           |32.44              |23.53            |8.91                 |
|Delta Air Lines Inc.        |281538           |29.55              |22.63            |6.92                 |
|Southwest Airlines Co.      |564396           |26.85              |21.14            |5.71                 |
|American Airlines Inc.      |244539           |34.23              |29.06            |5.16                 |
|JetBlue Airways             |101574           |37.51              |33.16            |4.35                 |
|Virgin Americ

In [12]:
# Prueba para ver qué está sucediendo con powerbi
from pyspark.sql.functions import col, length

# 1. Cargamos los datos crudos de Bronze
print("📂 Cargando vuelos...")
df_vuelos = spark.read.parquet("s3a://bronze/formatted/flights")

# 2. AUDITORÍA: Filtramos lo que NO cumple el estándar
# Buscamos códigos que NO tengan longitud 3 (es decir, los numéricos de 5 dígitos)
print("mag🔍 Buscando datos corruptos (Códigos numéricos)...")

df_errores = df_vuelos.filter(
    (length(col("ORIGIN_AIRPORT")) != 3) | 
    (length(col("DESTINATION_AIRPORT")) != 3)
)

# 3. RESULTADOS
cantidad = df_errores.count()
print(f"🚨 ¡ALERTA! Se han encontrado {cantidad} vuelos con códigos inválidos.")

if cantidad > 0:
    print("--- Muestra de los culpables ---")
    df_errores.select("YEAR", "MONTH", "AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT").show(10)
else:
    print("✅ ¡Sorpresa! No hay errores, los datos están limpios.")

📂 Cargando vuelos...
mag🔍 Buscando datos corruptos (Códigos numéricos)...
🚨 ¡ALERTA! Se han encontrado 486165 vuelos con códigos inválidos.
--- Muestra de los culpables ---
+----+-----+-------+--------------+-------------------+
|YEAR|MONTH|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|
+----+-----+-------+--------------+-------------------+
|2015|   10|     AA|         14747|              11298|
|2015|   10|     DL|         14771|              13487|
|2015|   10|     NK|         12889|              13487|
|2015|   10|     AA|         12892|              13303|
|2015|   10|     AA|         14771|              11057|
|2015|   10|     UA|         14771|              13930|
|2015|   10|     UA|         12892|              13930|
|2015|   10|     AA|         14869|              11057|
|2015|   10|     AA|         12892|              11298|
|2015|   10|     DL|         12892|              13487|
+----+-----+-------+--------------+-------------------+
only showing top 10 rows

